In [25]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_distances
from sklearn_extra.cluster import KMedoids
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import PorterStemmer
import string
import plotly.express as px

# Descargar recursos de nltk
nltk.download('stopwords')
nltk.download('words')

# Inicializar el stemmer y el conjunto de palabras válidas
stemmer = PorterStemmer()
valid_words = set(words.words())

# Función de preprocesamiento del texto
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    stop_words = set(stopwords.words('english'))  # Conjunto de stop words en inglés
    text = text.lower()  # Convertir el texto a minúsculas
    text = ''.join([char for char in text if char not in string.punctuation])  # Eliminar la puntuación
    words_list = text.split()
    words_list = [word for word in words_list if word not in stop_words]  # Eliminar stop words
    words_list = [stemmer.stem(word) for word in words_list]  # Aplicar stemming
    words_list = [word for word in words_list if word in valid_words]  # Eliminar palabras no válidas
    text = ' '.join(words_list)
    return text

# Leer el archivo CSV con codificación ISO-8859-1
def read_csv(file_path, num_rows=None):
    df = pd.read_csv(file_path, nrows=num_rows, encoding='ISO-8859-1')
    return df

# Convertir los textos a vectores TF-IDF y obtener la palabra más representativa
def get_top_word(text, vectorizer):
    tfidf_matrix = vectorizer.transform([text])  # Transformar el texto en una matriz TF-IDF
    feature_array = np.array(vectorizer.get_feature_names_out())  # Obtener los nombres de las características (palabras)
    tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]  # Ordenar los puntajes TF-IDF en orden descendente
    top_word = feature_array[tfidf_sorting][0]  # Obtener la palabra con el puntaje más alto
    return top_word

# Función principal para realizar el análisis PCA
def pca_analysis(file_path, num_rows=None, n_clusters=5):
    # Leer el archivo CSV
    df = read_csv(file_path, num_rows)
    
    # Asegurarse de que todas las entradas en la columna 'text' sean cadenas de texto
    df['text'] = df['text'].astype(str)
    
    # Preprocesar los textos
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    
    # Convertir los textos a vectores TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['cleaned_text']).toarray()
    
    # Obtener la palabra más representativa
    df['top_word'] = df['cleaned_text'].apply(lambda x: get_top_word(x, vectorizer))
    
    # Estandarizar los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Ajustar el número de clusters basado en el número de inliers
    n_clusters = min(n_clusters, X_scaled.shape[0])
    
    # Aplicar PCA con tres componentes
    pca = PCA(n_components=3)
    principal_components = pca.fit_transform(X_scaled)
    
    df_pca = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])
    df_pca['top_word'] = df['top_word'].values
    
    # Recalcular la matriz de distancias
    distance_matrix = cosine_distances(X_scaled)
    
    # Clustering con K-Medoids y distancia de similitud coseno
    kmedoids = KMedoids(n_clusters=n_clusters, metric='precomputed', random_state=42)
    clusters = kmedoids.fit_predict(distance_matrix)
    df_pca['cluster'] = clusters
    
    # Visualización interactiva en 3D con plotly
    fig = px.scatter_3d(df_pca, x='PC1', y='PC2', z='PC3', color='cluster', text='top_word',
                        title='PCA of Tweet Data in 3D', labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2', 'PC3': 'Principal Component 3'})
    
    fig.update_traces(marker=dict(size=5), selector=dict(mode='markers'))
    fig.update_layout(scene=dict(
                        xaxis_title='Principal Component 1',
                        yaxis_title='Principal Component 2',
                        zaxis_title='Principal Component 3'),
                      margin=dict(l=0, r=0, b=0, t=40))
    
    # Guardar el gráfico interactivo en un archivo HTML
    fig.write_html("pca_tweet_data.html")
    
    fig.show()

# Ejemplo de uso
file_path = 'C:/Users/nicol/OneDrive/MIA/Cursos/Aprendizaje Sup/Proyecto de Aplicacion/REPO GITHUB/proyecto-final-machine-learning/dataset/train.csv'  # Ruta actualizada del archivo CSV
num_rows = 1000  # Número de filas a considerar para el análisis, None para usar todas las filas
n_clusters = 5  # Número de clusters

pca_analysis(file_path, num_rows, n_clusters)






[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
